# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.multioutput import MultiOutputClassifier
import pickle
from sklearn import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Miriam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Miriam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# load data from database
engine = create_engine('sqlite:///ETL_disaster.db')
df_etl =pd.read_sql('SELECT * from ETL_disaster', engine)
X = df_etl['message']
Y = df_etl.drop(['id', 'message', 'genre', 'original'], axis = 1) #remove the columns without dummies

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
 pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [ ]:
pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

    # train classifier
pipeline.fit(X_train, y_train)




Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('Attribute: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))


Attribute: related

             precision    recall  f1-score   support

          0       0.60      0.43      0.50      1547
          1       0.83      0.91      0.87      4963
          2       0.88      0.16      0.27        44

avg / total       0.78      0.79      0.78      6554

Attribute: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5444
          1       0.82      0.38      0.51      1110

avg / total       0.87      0.88      0.86      6554

Attribute: offer



C:\Users\Miriam\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6533
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      1.00      6554

Attribute: aid_related

             precision    recall  f1-score   support

          0       0.69      0.91      0.79      3860
          1       0.76      0.43      0.55      2694

avg / total       0.72      0.71      0.69      6554

Attribute: medical_help

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6025
          1       0.50      0.01      0.02       529

avg / total       0.89      0.92      0.88      6554

Attribute: medical_products

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6250
          1       0.62      0.03      0.06       304

avg / total       0.94      0.95      0.93      6554

Attribute: search_and_rescue

             precision    recall

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def tuned_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    parameters = {'vect__min_df': [1, 3, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 3, 10]}

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 10)
    
    return cv

In [ ]:
model = tuned_model()
model.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
new_y_pred = model.predict(X_test)
new_y_pred_pd = pd.DataFrame(new_y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('Attribute: {}\n'.format(column))
    print(classification_report(y_test[column],new_y_pred_pd[column]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
cv.best_params_

In [ ]:
def build_model():
    pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutPutClassifier(SVC()))
    ])

    parameters = { 
        'features__text_pipeline__vect__max_df': (1.0),
        'features__text_pipeline__tfidf__use_idf': (True),
        'clf__estimator_kernel': ['poly'],
        'clf__estimator_degree': [1, 2, 3]
        
    }

    cv1 = GridSearchCV(pipeline2, param_grid=parameters)

    return cv1

In [ ]:
np.random.seed(81)
other_model = cv1.fit(X_train, Y_train)
 # predict on test data
y_pred1 = other_model.predict(X_test)

    # display results
display_results(y_test, y_pred1)

In [ ]:
print '\nClasification report:\n', classification_report(y_test, y_pred1)
print '\nConfussion matrix:\n',confusion_matrix(y_test, y_pred1)

In [ ]:
cv1.best_params_

### 9. Export your model as a pickle file

In [ ]:
new_pickle = open('disaster_reponse.sav', 'wb')
pickle.dump(tuned_model, new_pickle)
new_pickle.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.multioutput import MultiOutputClassifier
import pickle
from sklearn import metrics


# load data from database
def load_data(database_filepath):
    '''
    INPUT
        database_filepath - path where the database is stored
    OUTPUT
        X - Returns all features
        Y - data frame with all the numerical columns
        '''
    engine = create_engine('sqlite:///ETL_disaster.db')
    df_etl =pd.read_sql('SELECT * from ETL_disaster', engine)
    X = df_etl['message']
    Y = df_etl.drop(['id', 'message', 'genre', 'original'], axis = 1) #remove the columns without dummies
    return X, Y

def tokenize(text):
    '''
    INPUT
        text - text to be tokenized
    OUTPUT
        returns refined text that has been tokenized, all lower cased and lemmatized
    '''
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def tuned_model(X_train, y_train):
    '''
    INPUT
        X_train - data to be used as training features
        y_train - training data labels
    OUPUT
        returns a pipeline that is applied to grid search to create a new model
        
    '''
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    parameters = {'vect__min_df': [1, 3, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 3, 10]}

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 10)
   
    return cv
def model_testing(model, X_test, y_test):
    '''
    INPUT
        model - new model after training
        X_test - testing set data features
        y_test - testing set data labels
    OUTPUT
        returns the classification report 
    '''
    model.fit(X_train, y_train)     
    new_y_pred = model.predict(X_test)
    new_y_pred_pd = pd.DataFrame(new_y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('Attribute: {}\n'.format(column))
        print(classification_report(y_test[column],new_y_pred_pd[column]))
        
def save_data(model, new_model_filepath):
    '''
    INPUT
        model - the final model
        new_model_filepath - name for the new model
    OUTPUT
        save model as a pickle file
    '''
    new_pickle = open('new_model_filepath', 'wb')
    pickle.dump(model, new_pickle)
    new_pickle.close()
    
def main():
    if len(sys.argv) == 4:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y = load_data(database_filepath)
        X_train, X_test, y_train, y_test = train_test_split(X, Y)
        
       
        model = build_model(X_train,y_train)
        print('Building and Tuning model')
        
        print('Training the model')
        model = tuned_model()
        model.fit(X_train, y_train)
        
        print('Testing the new model and printing a classification report')
        model_testing(model, X_test, y_test)
        
   
        
        
        print('Model saved as MODEL: {}'.format(new_model_filepath))
        save_model(model, new_model_filepath)

        print('model saved!')


if __name__ == '__main__':
    main()